In [ ]:
#@title ⬅️ Mount Goole Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title ⬅️ Run ME (AIO Downaloder)

from IPython.display import clear_output
import time

def options():
  choices = ['Torrent', 'Seedr', 'BitPort', 'Mega', 'Others']
  num = 1
  for i in choices:
    print(f"{num}. {i}")
    num=num+1
  print('\n')
  sel = int(input("Enter your choice: "))
  clear_output
  eval(choices[(sel-1)]+"()")
  
def Torrent():
    from google.colab import files
    from IPython.display import display
    import ipywidgets as widgets
    !python -m pip install --upgrade pip setuptools wheel
    !python -m pip install lbry-libtorrent
    !apt install python3-libtorrent
    import libtorrent as lt
    
    ses = lt.session()
    ses.listen_on(6881, 6891)
    downloads = []
    clear_output()
    print("1. Magnet Link")
    print("2. Torrent File \n")
    sel = int(input("Enter your choice: "))
    if sel ==1:
        params = {"save_path": "/content/drive/Torrent"}

        while True:
            magnet_link = input("Enter Magnet Link Or Type Exit: ")
            if magnet_link.lower() == "exit":
                break
            downloads.append(
                lt.add_magnet_uri(ses, magnet_link, params)
            )
    elif sel == 2:
        while True:
            

            source = files.upload()
            params = {
                "save_path": "/content/drive/Torrent",
                "ti": lt.torrent_info(list(source.keys())[0]),
            }
            downloads.append(ses.add_torrent(params))

            moreFiles = input("Are there more files? Y/N ")
            if moreFiles.capitalize() == 'Y':
                None
            else:
                break
    
    
    state_str = [
        "queued",
        "checking",
        "downloading metadata",
        "downloading",
        "finished",
        "seeding",
        "allocating",
        "checking fastresume",
    ]
    
    layout = widgets.Layout(width="auto")
    style = {"description_width": "initial"}
    download_bars = [
        widgets.FloatSlider(
            step=0.01, disabled=True, layout=layout, style=style
        )
        for _ in downloads
    ]
    display(*download_bars)
    
    while downloads:
        next_shift = 0
        for index, download in enumerate(downloads[:]):
            bar = download_bars[index + next_shift]
            if not download.is_seed():
                s = download.status()
    
                bar.description = " ".join(
                    [
                        download.name(),
                        str(s.download_rate / 1000),
                        "kB/s",
                        state_str[s.state],
                    ]
                )
                bar.value = s.progress * 100
            else:
                next_shift -= 1
                ses.remove_torrent(download)
                downloads.remove(download)
                bar.close() # Seems to be not working in Colab (see https://github.com/googlecolab/colabtools/issues/726#issue-486731758)
                download_bars.remove(bar)
                print(download.name(), "complete")
        time.sleep(1)

def Seedr():
    !apt install axel
    clear_output()
    time.sleep(1)
    link = input("Enter the link: ")
    x = link.find('z')
    print(link[x:24])
    if link[x:24] == 'zip':
        !cd "/content/drive/Downloads" && axel -n 2 "$link" --output=tmp.zip
        !cd "/content/drive/Downloads" && unzip tmp.zip && rm tmp.zip 
    else:
        !cd "/content/drive/Downloads" && axel -n 4  "$link"

def BitPort():
    !apt install axel
    time.sleep(2)
    clear_output()
    link = input("Enter the link: ")
    if link[-3::] == 'zip':
        print('zip')

        !cd "/content/drive/Downloads" && axel -n 2 "$link" --output=tmp.zip
        !cd "/content/drive/Downloads" && unzip tmp.zip && rm tmp.zip 
    else:
        !cd "/content/drive/Downloads" && axel -n 4  "$link"

def Mega():
  !apt install megatools
  clear_output()
  time.sleep(2)
  link = input("Enter the link: ")
  #"https://mega.nz1wUiBQqZ#isXGprskZbLP4KnLNuNHcbI279s6FnLcsj8Vydm_sio"
  clear_output()
  print("No progress bar.")
  link = link.replace('#','!')
  link = link.replace('file/','#!')

  !cd "/content/drive/Downloads" && megadl "$link"


  

options()


In [ ]:
#@title ⬅️ To Change Download Location
location = "/content/drive"
print(f"Location is set to {location}")
if(input("Change location? (Y/N): ").lower() == 'y'):
    location = input("Enter the new location: ")

In [ ]:
#@title ⬅️ Mega to Drive
!cd /usr/local/bin and wget "https://github.com/aswindevsp/GoogleColab-AIO-Downloader/raw/main/res/megatools"
link = []

if 'location' not in globals(): 
    location = "/content/drive"

while(True):
    temp = input("Enter the link or 'exit' to stop: "))
    if(temp.lower() == 'exit'):
        break;
    else:
        link.append(temp)

for i in link:
    !cd location && megatools dl $link[i] 
    